In [ ]:
"""
A speed comparison of methods for matrix multiplication:
    Numpy
    Scipy.linalg.blas
    PyOpenCl
"""

In [ ]:
def timer(func):
    def timer_wrap(*args, **kwargs):
        start = datetime.now()
        func(*args,**kwargs)
        stop = datetime.now()
        print( (stop-start).microseconds)
    return timer_wrap


In [118]:
import numpy as np
from datetime import datetime

import pyopencl as cl
from pyopencl.tools import get_test_platforms_and_devices
import numpy as np

import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [119]:
I_ext = np.ones(shape=(2048,2048))
I_90 = np.ones(shape=(2048,2048))
I_135 = np.ones(shape=(2048,2048))
I_45 = np.ones(shape=(2048,2048))
I_0 = np.ones(shape=(2048,2048))

(n, m, p) = (2048, 2048, 2048)

# a = np.random.randn(n, m).astype(np.float32)
# b = np.random.randn(m, p).astype(np.float32)
a = np.random.randint(2, size=(n*m))
b = np.random.randint(2, size=(m*p))
c = np.random.randint(2, size=(m*p))
d = np.random.randint(2, size=(m*p))
e = np.random.randint(2, size=(m*p))

a = np.reshape(a, (2048,2048)).astype(np.float32)
b = np.reshape(b, (2048,2048)).astype(np.float32)
c = np.reshape(c, (2048,2048)).astype(np.float32)
d = np.reshape(d, (2048,2048)).astype(np.float32)
e = np.reshape(e, (2048,2048)).astype(np.float32)

In [120]:
img_raw = np.stack((I_ext, I_0, I_45, I_90, I_135))
img_raw_ab = np.stack((a,b,c,d,e))

In [121]:
img_raw.shape
img_raw_ab.shape

In [122]:
start = datetime.now()
img_raw_flat = np.reshape(img_raw, (5, 2048*2048))
stop = datetime.now()
print((stop-start).microseconds)

start = datetime.now()
img_raw_flat_ab = np.reshape(img_raw_ab, (5, 2048*2048))
stop = datetime.now()
print((stop-start).microseconds)

75
6035


In [123]:
def compute_inst_matrix():
    chi = 0.2
    inst_mat = np.array([[1, 0, 0, -1],
                         [1, np.sin(chi), 0, -np.cos(chi)],
                         [1, 0, np.sin(chi), -np.cos(chi)],
                         [1, -np.sin(chi), 0, -np.cos(chi)],
                         [1, 0, -np.sin(chi), -np.cos(chi)]])

    inst_mat_inv = np.linalg.pinv(inst_mat)
    return inst_mat_inv.astype(np.float32)

In [124]:
start = datetime.now()

inverse = compute_inst_matrix()

stop = datetime.now()
print((stop-start).microseconds)

460


In [125]:
inverse.shape
inverse.strides
type(inverse[0][0])

img_raw_flat.shape
img_raw_flat.strides
type(img_raw_flat[0][0])

img_raw_flat_ab.shape
img_raw_flat_ab.strides
type(img_raw_flat_ab[0][0])


(4, 5)

(20, 4)

numpy.float32

(5, 4194304)

(33554432, 8)

numpy.float64

(5, 4194304)

(16777216, 4)

numpy.float32

In [127]:
#============= timing test =====================
# data is float 64

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)


36262


In [128]:

#============= timing test =====================
# data is float 32

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat_ab)
stop = datetime.now()
print((stop-start).microseconds)

36262


In [ ]:
inverse.flags
img_raw_flat.flags
np.show_config()


In [110]:

#=========================================================
#=============== Scipy Blas ==============================
#=========================================================

In [111]:
from scipy.linalg.blas import dgemm
from scipy.linalg.blas import dsymm


In [26]:
start = datetime.now()
img_stokes_flat = dgemm(alpha=1., a=inverse, b=img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)

239713


In [ ]:
start = datetime.now()
img_stokes_flat = dsymm(alpha=1., a=inverse, b=img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)


In [ ]:
import numpy.distutils.system_info as sysinfo
sysinfo.get_info('blas')

In [68]:
np.__config__.show()

mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/Users/bryant.chhun/anaconda3/envs/InstantPol3.6/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAV

In [74]:
#=========================================================
#=============== begin openCL tests ======================
#=========================================================


In [ ]:
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'
os.environ['PYOPENCL_CTX'] = '1'

(n, m, p) = (2048, 2048, 2048)

I_ext = np.ones(shape=(2048,2048))
I_90 = np.ones(shape=(2048,2048))
I_135 = np.ones(shape=(2048,2048))
I_45 = np.ones(shape=(2048,2048))
I_0 = np.ones(shape=(2048,2048))

a = np.random.randint(2, size=(n*m))
b = np.random.randint(2, size=(m*p))
c = np.random.randint(2, size=(m*p))
d = np.random.randint(2, size=(m*p))
e = np.random.randint(2, size=(m*p))
out = 

a = np.random.randint(2, size=(n*m))
b = np.random.randint(2, size=(m*p))
c = np.zeros((n*p), dtype=np.float32)

a = np.reshape(a, (2048,2048)).astype(np.float64)
b = np.reshape(b, (2048,2048)).astype(np.float64)
c = np.reshape(c, (2048,2048)).astype(np.float64)
d = np.reshape(d, (2048,2048)).astype(np.float64)
e = np.reshape(e, (2048,2048)).astype(np.float64)

a = a.astype(np.float32)
b = b.astype(np.float32)


In [130]:
img_raw = np.stack((I_ext, I_0, I_45, I_90, I_135))
img_raw_ab = np.stack((a,b,c,d,e))

inverse = compute_inst_matrix()

img_raw_flat = np.reshape(img_raw, (5, 2048*2048))
img_raw_flat_ab = np.reshape(img_raw_ab, (5, 2048*2048))



In [131]:
platform = cl.get_platforms()
my_gpu_devices = platform[0].get_devices(device_type=cl.device_type.GPU)
my_gpu_devices
# ctx = cl.Context(devices=my_gpu_devices)


[<pyopencl.Device 'Intel(R) HD Graphics 630' on 'Apple' at 0x1024500>,
 <pyopencl.Device 'AMD Radeon Pro 560 Compute Engine' on 'Apple' at 0x1021c00>]

In [132]:
my_gpu_devices = platform[0].get_devices(device_type=cl.device_type.GPU)
ctx = cl.Context(devices=my_gpu_devices)

queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_buf = cl.Buffer\
   (ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=inverse)
b_buf = cl.Buffer\
   (ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=img_raw_flat_ab)
c_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)

In [133]:
prg = cl.Program(ctx, """
    __kernel void multiply(ushort n,
                            ushort m, 
                            ushort p, 
                            __global float *a,
                            __global float *b, 
                            __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = 0.0f;
      int rowC = gid/p;
      int colC = gid%p;
      __global float *pA = &a[rowC*m];
      __global float *pB = &b[colC];
      for(int k=0; k<m; k++)
      {
         pB = &b[colC+k*p];
         c[gid] += (*(pA++))*(*pB);
      }
    }
    """).build()

In [134]:
inverse = inverse.astype(np.float32)
img_raw_flat = img_raw_flat.astype(np.float32)

start = datetime.now()
img_stokes_flat = np.dot(inverse, img_raw_flat)
stop = datetime.now()
print((stop-start).microseconds)


time is = 4159


matrix A:
[[1. 0. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 1. 0.]
 ...
 [1. 0. 1. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
matrix B:
[[1. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 ...
 [1. 1. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 1. 1.]]
multiplied A*B:
[[253.48325 321.19202 210.37476 ... 190.34973 320.70874 257.54175]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 ...
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]]


In [135]:
start = datetime.now()
prg.multiply(queue, c.shape, None, np.uint16(n), np.uint16(m), np.uint16(p), a_buf, b_buf, c_buf)
stop = datetime.now()
print("time is = "+str((stop-start).microseconds))

a_mul_b = np.empty_like(c)
cl.enqueue_copy(queue, a_mul_b, c_buf)

print("matrix A:")
print(a.reshape(n, m))
print("matrix B:")
print(b.reshape(m, p))
print("multiplied A*B:")
print(a_mul_b.reshape(n, p))


time is = 4159


matrix A:
[[1. 0. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 1. 0.]
 ...
 [1. 0. 1. ... 1. 0. 0.]
 [1. 0. 0. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 0. 0.]]
matrix B:
[[1. 0. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 ...
 [1. 1. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 1. 1.]]
multiplied A*B:
[[253.48325 321.19202 210.37476 ... 190.34973 320.70874 257.54175]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 ...
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]]
